# DATA 440 Final Project
# Part II: Data Preprocessing


Overall Function: Pass in raw NBA Box Score data to clean_data() function and return dataframe with cleaned data

Functions:
1. Create target variable, which shifts the 'won' column so that it gives the outcome of the team's next game (Binary)
    - Note: Grouped by team and season
    - Ex: If Detroit Piston (DET) won their second game in the 2016 season, then with respect to the row that corresponds to their first game, the 'target' value will be equal to 1
2. Move meta-data columns (team name, opposing team, season, etc.) to front of the DataFrame
3. Remove maximum individual statistics of each game for each team
4. Drop duplicate rows; keep rows so that first team is the Home team
5. Write new DataFrame to csv file, used in subsequent notebooks

### 1) Imports


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

/Users/Harry/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


---
### 2) Functions

In [19]:
def create_target(team):
    team['target'] = team['won'].shift(-1,axis=0) # Get the values of the 'won' column 1 period ahead (-1 in this case)
    return team
    
def clean_data(data):
    '''
        This function takes the NBA box score data as the input and 
        returns a cleaned-up dataframe.
        
        
    '''
    data = data.sort_values(['season','date']).reset_index(drop=True)

    cols_to_move = ['team','total','team_opp','total_opp','season','date','home'] # Move team name columns to beginning of dataframe
    for idx,col in enumerate(cols_to_move): 
        col_to_pop = data.pop(col)
        data.insert(idx,col,col_to_pop)
    
    # Dropping all unnecessary columns
    cols_to_drop = ['mp','mp_opp','mp.1','mp_opp.1','+/-','+/-_opp','mp_max','mp_max.1',
                   'mp_max_opp','mp_max_opp.1','usg%','usg%_opp','total','total_opp','home_opp','index_opp'] # Drop unnecessary columns
    cols_to_drop = [c for c in data.columns if c in cols_to_drop]# Make sure these columns are in the data
    
    data = data.drop(cols_to_drop,axis=1)
    data = data.reset_index(drop=True) # Reset index of dataframe
    
    
    team_stats = [col for col in data.columns if '_max' not in col] # Remove individual statistics (i.e. max stats)
    data = data[team_stats]
    
    # Add new target
    data = data.groupby(['team','season'],group_keys=False).apply(create_target)
    data['target'].fillna(-1, inplace=True) # use placeholder of -1 for now
    data['target'] = data['target'].astype(int)
    
      # Drop Duplicate Rows: Keep Rows where Home == 1
    data = data[data['home'] == 1].reset_index(drop=True)
    data = data.rename(columns={'team':'Home Team','team_opp':'Away Team'}) # Rename team and team_opp
    data = data.drop(['home'],axis=1) #No need for home column anymore
    
    if 'Unnamed: 0' in data.columns:
        data = data.drop('Unnamed: 0', axis=1)
    return data

def create_first_cleaned_data(cleaned_data):
    '''
        Establish first cleaned dataframe of NBA data from 2016-2022
    '''
    cleaned_data.to_csv('../data/cleaned_nba_data.csv', index=False)
    return cleaned_data

def add_new_data(new_data):
    '''
        Add new box score data to the dataframe stored as csv
    '''
    original_data = pd.read_csv('../data/cleaned_nba_data.csv')
    nba_clean_data = pd.concat([original_data,new_data], axis=0).reset_index(drop=True)
    nba_clean_data.to_csv('../data/cleaned_nba_data.csv', index=False)# Save to dataframe
    return nba_clean_data

---
### 3) Execute


In [20]:
# Load in 2016-2022 data and 2023 data
nba_data_2016_2022 = pd.read_csv('../data/nba_games.csv')
nba_data_2023 = pd.read_csv('../data/nba_games_2023.csv')

nba_clean_data_2016_2022 = clean_data(nba_data_2016_2022) # Cleaned 2016-2022
nba_clean_data = create_first_cleaned_data(nba_clean_data_2016_2022) # first iteration
nba_clean_data_23 = clean_data(nba_data_2023)
nba_clean_data = add_new_data(nba_clean_data_23) # Combine to get cleaned 2016-2023

In [21]:
nba_clean_data

,Home Team,Away Team,season,date,fg,fga,fg%,3p,3pa,3p%,...,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp,won,target
0,ATL,DET,2016,2015-10-27,37.0,82.0,0.451,8.0,27.0,0.296,...,83.7,59.6,62.2,5.2,5.5,12.3,111.2,98.6,False,1
1,GSW,NOP,2016,2015-10-27,41.0,96.0,0.427,9.0,30.0,0.300,...,54.3,37.1,60.0,9.0,4.5,15.9,94.9,110.9,True,1
2,CHI,CLE,2016,2015-10-27,37.0,87.0,0.425,7.0,19.0,0.368,...,84.8,51.5,68.4,5.0,10.3,9.0,95.5,97.5,True,1
3,BRK,CHI,2016,2015-10-28,39.0,93.0,0.419,0.0,9.0,0.000,...,68.0,46.4,47.6,9.1,3.6,18.0,116.1,101.0,False,0
4,TOR,IND,2016,2015-10-28,36.0,80.0,0.450,7.0,18.0,0.389,...,78.0,44.4,71.9,13.5,4.8,11.5,95.5,102.3,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10201,DEN,MIA,2023,2023-06-01,40.0,79.0,0.506,8.0,27.0,0.296,...,84.2,48.9,66.7,5.5,7.7,7.6,102.3,114.4,True,0
10202,DEN,MIA,2023,2023-06-04,39.0,75.0,0.520,11.0,28.0,0.393,...,71.9,44.9,73.7,5.8,8.5,11.2,127.8,124.3,False,1
10203,MIA,DEN,2023,2023-06-07,34.0,92.0,0.370,11.0,35.0,0.314,...,81.8,63.7,68.3,3.3,8.8,12.4,120.4,103.8,False,0
10204,MIA,DEN,2023,2023-06-09,35.0,78.0,0.449,8.0,25.0,0.320,...,78.4,47.9,66.7,12.4,13.2,6.4,121.9,107.2,False,0
